In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.metrics import accuracy_score
from textblob import TextBlob
from scipy.sparse import hstack, vstack
from scipy.sparse import coo_matrix, csr_matrix
from tpot import TPOTClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import random
import nltk
from sklearn.preprocessing import MaxAbsScaler, QuantileTransformer
from sklearn.neural_network import MLPClassifier



scaler = StandardScaler()
robust_scaler = RobustScaler()

# Carica il tuo DataFrame da 'train_B_text.csv'
dataset = pd.read_csv('train_B_text.csv')
simbol='"'
dataset['Counter quotmarks'] = dataset['Title'].str.count(simbol)
def sentiment_calculator(title):
    analysis = TextBlob(title)
    if analysis.sentiment.polarity > 0:
        return 1 #positive
    elif analysis.sentiment.polarity == 0:
        return 0 #neutral
    else:
        return -1 #negative
    
dataset['Count words'] = dataset['Title'].apply(lambda x: len(str(x).split()))

#dataset = dataset.drop('Id', axis=1)

    
def mean_length(title, n_words):
    words = title.split() 
    total_length = sum(len(word) for word in words)
    mean_length = total_length / n_words
    return mean_length

dataset["Mean length"] = dataset.apply(lambda row: mean_length(row["Title"], row["Count words"]), axis=1)


punctuation_symbols = [',', '.', ';',"'",":",'%','-']
for i, symbol in enumerate(punctuation_symbols):
    table_name = f'symbol_{i}'  # Nome della tabella con indice i
    dataset[table_name] = dataset['Title'].str.count(symbol)

dataset['Starts_with_quotmarks'] = dataset['Title'].str.startswith('"').astype(int)


def count_number(title):
    number = re.findall(r'\d+', title)
    return len(number)
dataset["Numbers"]=dataset["Title"].apply(count_number)



dataset['Sentiment'] = dataset['Title'].apply(sentiment_calculator)
dataset['Capital words'] = dataset['Title'].apply(lambda x: len([word for word in x.split() if word.istitle()]))
dataset['Difference']=dataset['Count words']-dataset['Capital words']

dataset['Count lword'] = dataset['Title'].apply(lambda x: len([word for word in x.split() if len(word)>2]))

dataset['Difference2']=dataset['Count lword']-dataset['Capital words']

# Estrai le colonne 'Title' e 'Fake/Real' dal DataFrame
titles = dataset['Title']
labels = dataset['Fake/Real']

# Converti le etichette in 0 e 1
labels = labels.map({'real': 1, 'fake': 0})

corpus = dataset['Title'].tolist()
sentences = [word_tokenize(title) for title in corpus]

model = Word2Vec(sentences, vector_size=50, window=5, min_count=1)

features_word2vec = []
for sentence in sentences:
    sentence_features = [model.wv[word] for word in sentence if word in model.wv]
    if sentence_features:
        features_word2vec.append(np.mean(sentence_features, axis=0)) 
    else:
        features_word2vec.append(np.zeros(50)) 


# Converti 'features_word2vec' in un array numpy
word2vec_array = np.array(features_word2vec)

document_index = 0  
cosine_similarities = cosine_similarity([word2vec_array[document_index]], word2vec_array)
distance_metric = cosine_similarities[0]

dataset['Cos similarity'] = distance_metric

tfidf_vectorizer = TfidfVectorizer()

tfidf_matrix = tfidf_vectorizer.fit_transform(dataset['Title'])


def has_acronym(title):
    words = title.split()  
    for word in words:
        if word.isupper() and len(word) > 1:
            return 1 
    return 0  

dataset['Has Acronym'] = dataset['Title'].apply(has_acronym)

X_combined =hstack((tfidf_matrix,dataset['Id'].values.reshape(-1, 1),dataset['Counter quotmarks'].values.reshape(-1, 1),
                    dataset["Mean length"].values.reshape(-1,1),dataset['Capital words'].values.reshape(-1,1),dataset['Difference2'].values.reshape(-1,1)
                    ,dataset['Starts_with_quotmarks'].values.reshape(-1,1),
                    dataset['symbol_0'].values.reshape(-1, 1),dataset['symbol_1'].values.reshape(-1, 1),dataset['symbol_2'].values.reshape(-1, 1),  
                     dataset['symbol_3'].values.reshape(-1, 1),dataset['symbol_4'].values.reshape(-1, 1),  
                     dataset['symbol_5'].values.reshape(-1, 1),dataset['symbol_6'].values.reshape(-1, 1),dataset['Has Acronym'].values.reshape(-1,1)
                    ))


In [10]:
import xgboost as xgb
from xgboost import XGBClassifier
datatest=pd.read_csv('test_B_text.csv')
simbol='"'
datatest['Counter quotmarks'] = datatest['Title'].str.count(simbol)
    
datatest['Count words'] = datatest['Title'].apply(lambda x: len(str(x).split()))

    

datatest["Mean length"] = datatest.apply(lambda row: mean_length(row["Title"], row["Count words"]), axis=1)


punctuation_symbols = [',', '.', "'",":"]
datatest['punctuation'] = datatest['Title'].apply(lambda x: sum(x.count(symbol) for symbol in punctuation_symbols))


datatest["Numbers"]=datatest["Title"].apply(count_number)
datatest['Count lword'] = datatest['Title'].apply(lambda x: len([word for word in x.split() if len(word)>2]))


datatest['Sentiment'] = datatest['Title'].apply(sentiment_calculator)
datatest['Capital words'] = datatest['Title'].apply(lambda x: len([word for word in x.split() if word.istitle()]))


datatest['Difference2']=datatest['Count lword']-datatest['Capital words']


punctuation_symbols = [',', '.', ';',"'",":",'%','-']
for i, symbol in enumerate(punctuation_symbols):
    table_name = f'symbol_{i}'  # Nome della tabella con indice i
    datatest[table_name] = datatest['Title'].str.count(symbol)

datatest['Starts_with_quotmarks'] = datatest['Title'].str.startswith('"').astype(int)




corpus = datatest['Title'].tolist()


sentences = [word_tokenize(title) for title in corpus]
model = Word2Vec(sentences, vector_size=50, window=5, min_count=1)

features_word2vec = []
for sentence in sentences:
    sentence_features = [model.wv[word] for word in sentence if word in model.wv]
    if sentence_features:
        features_word2vec.append(np.mean(sentence_features, axis=0)) 
    else:
        features_word2vec.append(np.zeros(50)) 


# Converti 'features_word2vec' in un array numpy
word2vec_array = np.array(features_word2vec)
document_index = 0  
cosine_similarities = cosine_similarity([word2vec_array[document_index]], word2vec_array)
distance_metric = cosine_similarities[0]
datatest['Cos similarity'] = distance_metric


tfidf_matrix = tfidf_vectorizer.transform(datatest['Title'])

def has_acronym(title):
    words = title.split()  
    for word in words:
        if word.isupper() and len(word) > 1:
            return 1 
    return 0  

datatest['Has Acronym'] = datatest['Title'].apply(has_acronym)

X_combined_test=hstack((tfidf_matrix,datatest['Id'].values.reshape(-1, 1),datatest['Counter quotmarks'].values.reshape(-1, 1),datatest['Mean length'].values.reshape(-1,1),
                    datatest['Capital words'].values.reshape(-1,1),datatest['Difference2'].values.reshape(-1,1)
                    ,datatest['Starts_with_quotmarks'].values.reshape(-1,1),datatest['symbol_0'].values.reshape(-1, 1),datatest['symbol_1'].values.reshape(-1, 1),
                    datatest['symbol_2'].values.reshape(-1, 1), datatest['symbol_3'].values.reshape(-1, 1),datatest['symbol_4'].values.reshape(-1, 1),  
                     datatest['symbol_5'].values.reshape(-1, 1),datatest['symbol_6'].values.reshape(-1, 1),datatest['Has Acronym'].values.reshape(-1, 1)
                    ))


#tpot = TPOTClassifier(generations=3, population_size=20, random_state=42, verbosity=2)

#best_model.fit(csr_matrix(X_combined),labels)
#best_model=MLPClassifier(hidden_layer_sizes=(50, 50), alpha=0.0001, learning_rate_init=0.001)
best_model = RandomForestClassifier(bootstrap=True, criterion='entropy', max_features=0.95, min_samples_leaf=1, min_samples_split=6, n_estimators=100)


best_model.fit(X_combined, labels)


predictions = best_model.predict(X_combined_test)

Y_pred_testB = pd.Series(np.where(predictions==1, 'real', 'fake'))
Y_pred_testB.to_csv('Y_pred_testB.10.csv')

In [11]:
gabriela= pd.read_csv('y_prediction_v20.csv').iloc[:256,-1]
y_prediction_v39=pd.concat([gabriela,pd.read_csv('Y_pred_testB.10.csv').iloc[:,-1]]).reset_index(drop=True).reset_index()
y_prediction_v39.columns = ['Id','Prediction']
y_prediction_v39.to_csv('y_prediction_v39.csv', sep=',',index=False)